In [79]:
from pymongo import MongoClient
from bson import ObjectId
import urllib.parse
from sqlalchemy import create_engine, Column, BigInteger, Integer, Float, String, Enum, ForeignKey, Table, MetaData
from sqlalchemy.sql.sqltypes import DateTime, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.sql.ddl import CreateTable
from sqlalchemy.dialects import postgresql
import re

In [80]:
db_config = {
    'postgres': {
    'host': 'localhost',
    'database': 'tenshi',
    'username': 'postgres',
    'password': 'postgres',
    },
    'mongo': {
        'host': 'localhost',
        'port': '27017',
        'database': 'tenshi',
        'username': 'root',
        'password': 'root',
        'auth_source': 'admin'
    }
}

In [ ]:
def get_mongo_connection(mongo_config):
    """
    Create a MongoDB connection string from the configuration dictionary.
    """
    username = urllib.parse.quote_plus(mongo_config['username'])
    password = urllib.parse.quote_plus(mongo_config['password'])
    connection_string = (f"mongodb://{username}:{password}@"
                         f"{mongo_config['host']}:{mongo_config['port']}/"
                         f"{mongo_config['database']}?authSource={mongo_config['auth_source']}")
    return MongoClient(connection_string)

def get_postgres_engine(postgres_config):
    username = urllib.parse.quote_plus(postgres_config['username'])
    password = urllib.parse.quote_plus(postgres_config['password'])
    connection_string = (f"postgresql+psycopg2://{username}:{password}@"
                         f"{postgres_config['host']}/"
                         f"{postgres_config['database']}")
    return create_engine(connection_string, echo=True)
    
# Function to convert camel case to snake case
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()
    # Then, remove any non-alphanumeric characters (except underscores)
    # by keeping only characters that are alphanumeric or underscores
    name = re.sub(r'[^a-zA-Z0-9_]', '', name)
    return name


In [78]:
def process_documents(documents):
    """
    Process each document (e.g., generate SQL and print it).
    """

    # A dictionary to hold all your table classes, since they're dynamically created
    table_classes = {}

    # A list to hold relations for alter table statements
    relations_for_alter = []

    # Base class for declarative tables
    Base = declarative_base()

    # Create a PostgreSQL table for each MongoDB document
    for doc in documents:
        table_name = camel_to_snake(doc['externalId'])
        class_name = camel_to_snake(table_name)

        primary_key_column_name = f"{table_name}_id"  # format the primary key as 'table_name_id'
        # Create a new type dynamically
        table_class = type(class_name, (Base,), {
            '__tablename__': table_name,
            primary_key_column_name: Column(BigInteger, primary_key=True),
            '__table_args__': {'extend_existing': True}  # Allow redefinition if rerunning
        })
    
        # Add properties as columns
        for prop in doc['properties']:
            col_name = camel_to_snake(prop['externalId'])
            col_type = String  # Default type is String
    
            if prop['inputType'] == 'NUMBER':
                col_type = Float
            elif prop['inputType'] == 'DATE_TIME':
                col_type = DateTime
            elif prop['inputType'] == 'DATE':
                col_type = Date
            elif 'options' in prop:  # Enum type
                # assuming the dictionaries have a 'name' field you want to use for the Enum
                option_names = [option['displayName'] for option in prop['options']]
                col_type = Enum(*option_names, name=f"{table_name}_{col_name}_enum")
                
            comment = f"{prop['_id']}"
            setattr(table_class, col_name, Column(col_type, comment=comment))
    
        # Save the table class to the dictionary
        table_classes[table_name] = table_class
    
    # Now handle relations
    for doc in documents:
        for rel in doc.get('relations', []):
            # Find the related table
            related_table_name = rel['externalId']
            related_table_class = table_classes.get(related_table_name)
    
            if related_table_class:
                # Prepare the information needed for alter table statements
                source_table_name = doc['externalId']
                source_column_name = f"{related_table_name}_id"
                target_table_name = related_table_name
                target_column_name = 'id'  # assuming the primary key column is named 'id'
                relations_for_alter.append({
                    'source_table': source_table_name,
                    'source_column': source_column_name,
                    'target_table': target_table_name,
                    'target_column': target_column_name
                })
    
    # Print the SQL statements for creating tables
    for table_class in table_classes.values():
        table = CreateTable(table_class.__table__).compile(dialect=postgresql.dialect())
        print(str(table))
    
    # Print the SQL statements for alter table add foreign key constraints
    for rel in relations_for_alter:
        alter_stmt = (
            f"ALTER TABLE {rel['source_table']} "
            f"ADD CONSTRAINT fk_{rel['source_table']}_{rel['source_column']} "
            f"FOREIGN KEY ({rel['source_column']}) "
            f"REFERENCES {rel['target_table']}({rel['target_column']});"
        )
        print(alter_stmt)
        
client = get_mongo_connection(db_config['mongo'])
db = client[db_config['mongo']['database']]
collection = db['objectTypes']
documents = collection.find({'usageStatus': 1}).sort('externalId', 1)
process_documents(documents)


CREATE TABLE accessories (
	accessories_id BIGSERIAL NOT NULL, 
	external_id VARCHAR, 
	display_name VARCHAR, 
	created_at TIMESTAMP WITHOUT TIME ZONE, 
	updated_at TIMESTAMP WITHOUT TIME ZONE, 
	created_by VARCHAR, 
	updated_by VARCHAR, 
	usage_status FLOAT, 
	PRIMARY KEY (accessories_id)
)



CREATE TABLE ad_instruments_or_equipments (
	ad_instruments_or_equipments_id BIGSERIAL NOT NULL, 
	display_name ad_instruments_or_equipments_display_name_enum, 
	external_id ad_instruments_or_equipments_external_id_enum, 
	created_at TIMESTAMP WITHOUT TIME ZONE, 
	updated_at TIMESTAMP WITHOUT TIME ZONE, 
	created_by VARCHAR, 
	updated_by VARCHAR, 
	usage_status FLOAT, 
	ad_instrument_or_equipment_status ad_instruments_or_equipments_ad_instrument_or_equipment_status_enum, 
	make ad_instruments_or_equipments_make_enum, 
	model ad_instruments_or_equipments_model_enum, 
	qualification_done_on DATE, 
	qualification_due_on DATE, 
	PRIMARY KEY (ad_instruments_or_equipments_id)
)



CREATE TABLE areas 

/tmp/ipykernel_30353/4274175832.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
client = get_mongo_connection(db_config['mongo'])
db = client[db_config['mongo']['database']]
collection = db['objectTypes']
documents = collection.find({'usageStatus': 1}).sort('externalId', 1)
process_documents(documents)

In [ ]:
import json

documents = collection.find({'usageStatus': 1}).sort('externalId', 1)


def snake_case(name):
    return name.lower().replace(' ', '_')

def is_foreign_key(prop):
    return prop['relation'] is not None

def get_ref_table(prop):
    return prop['relation']['target']['externalId']

tables = {}
relations = []
for doc in list(documents):

    # obj = json.loads(doc)
    obj = doc

    table_name = obj['externalId']
    columns = []

    for prop in obj['properties']:
        if 'relation' in prop:
            fk_table = prop['relation']['target']['externalId']
            fk_column = snake_case(prop['displayName']) + "_id"
            columns.append(fk_column)
            relations.append((table_name, fk_table, fk_column))
        else:
            column = snake_case(prop['displayName'])
            columns.append(column)

    tables[table_name] = columns

print("-- DDL SQL --")

for table_name, columns in tables.items():

    print(f"CREATE TABLE {table_name} (")

    column_strings = [f"  {c} VARCHAR(255)" for c in columns]
    column_strings.append("  PRIMARY KEY (id)")

    print(",\n".join(column_strings))
    print(");")
    print()

for src_table, ref_table, fk_col in relations:

    pk = [c for c in tables[ref_table] if c.endswith('_id')][0]

    print(f"ALTER TABLE {src_table} ADD CONSTRAINT fk_{src_table}_{ref_table}")
    print(f"  FOREIGN KEY ({fk_col}) REFERENCES {ref_table} ({pk});")
    print()